<div style="text-align:center">
    <h2> CS3100 - Lecture 09 - Wed, Aug 18, 10am </h2>
    <h1> Even More on Functions </h1>
</div>

## Recursive Functions

Recursive functions can call themselves. The syntax for recursive function definition is:

```ocaml
let rec foo x = ...
```

Notice the `rec` key word.

## Recursive Functions

In [56]:
let rec findsum n = 
  if n <= 0 then 0
  else n + findsum (n-1)

val findsum : int -> int = <fun>


In [57]:
findsum 5

- : int = 15


## Recursing too deep

Let's invoke `findsum` with larger numbers. 

In [58]:
findsum 1000000

error: runtime_error

## Stack buildup

```ocaml
let rec findsum n = 
  if n <= 0 then 0
  else n + findsum (n-1)
```

Some work `"+ n"` left to do after the recursive call returns. This builds up stack frames.

## Stack buildup

For `findsum 5`:

* `5 + (findsum 4)`
* `4 + (findsum 3)`
* `3 + (findsum 2)`
* `2 + (findsum 1)`
* `1 + (findsum 0)`
* the number `n` kept in stack frame.
* This causes stackoverflow for small values itself.


## Tail recursion

Rewrite the function such that the recursive call is the last thing that the function does:

In [60]:
let rec findsum_tailrec acc n = 
  if n <= 0 then acc
  else findsum_tailrec (n + acc) (n-1)
  
let findsum_tailrec n = findsum_tailrec 0 n

val findsum_tailrec : int -> int -> int = <fun>


val findsum_tailrec : int -> int = <fun>


In [62]:
findsum_tailrec 1000000 5

error: compile_error

## Tail recursion

```ocaml
let rec findsum_tailrec n acc = 
  if n <= 0 then acc
  else findsum_tailrec (n-1) (n + acc)
```

* No work left to do when the recursive call returns except return result to caller.
* OCaml compiler does **tail call optimisation** that pops current call frame before invoking recursive call. 
  + No stack buildup => equivalent to writing a tight loop. 

## Labelled and Optional Arguments in Functions

* type and name of a function - good indicators of what the arguments should be.

In [63]:
String.sub

- : string -> int -> int -> string = <fun>


* OCaml supports labeled arguments to functions.

```Ocaml
let f ~name1:arg1 ~name2:arg2 = arg1 + arg2
```

* Application can use names of arguments also.

```Ocaml
f ~name2:3 ~name1:4
```

In [64]:
let f ~xvalue:x ~yvalue:y = x + 2*y

val f : xvalue:int -> yvalue:int -> int = <fun>


In [67]:
f 5 10

- : int = 25


* We can make some arguments as optional for the function

In [75]:
let newf ?xvalue:(x:int=8) (y:int) = x + 2*y

val newf : ?xvalue:int -> int -> int = <fun>


In [71]:
newf ~xvalue:5 4

- : int = 13


In [72]:
newf 5

- : int = 18


## Operators as Functions

In [77]:
( +. )

- : float -> float -> float = <fun>


In [78]:
let add x y = (+) x y

val add : int -> int -> int = <fun>


In [79]:
( * )

- : int -> int -> int = <fun>


In [ ]:
let ( ^^ ) x y = max x y

In [ ]:
2 ^^ 3

## Writing Comments - Pre-conditions and Post-conditions


```Ocaml
(** [lowercase_ascii c] is the lowercase ASCII equivalent of character [c]. *)

(** [index s c] is the index of the first occurrence of character [c] in string [s].  
    Raises: [Not_found] if [c] does not occur in [s]. *)

(** [random_int bound] is a random integer between 0 (inclusive) and [bound] (exclusive).
    Requires: [bound] is greater than 0 and less than 2^30. *)
```

## Pattern Matching

In [80]:
let f x = 
    if x = "foo"
    then "it is foo"
    else if x = "bar"
    then "bar is the one"
    else if x = "zap"
    then "totally different"
    else "our default: " ^ x

val f : string -> string = <fun>


In [85]:
(f "abc")

- : string = "our default: abc"


In [86]:
let newf x =
    match x with
      "foo" -> "it is foo"
    | "bar" -> "bar is the one"
    | "zap" -> "totally different"
    |  _    -> "our default: " ^ x

val newf : string -> string = <fun>


In [87]:
let rec fact n =
    match n with
      0 -> 1
    | _ -> n*(fact (n-1))

val fact : int -> int = <fun>


In [88]:
(fact 5)

- : int = 120


## Pattern Matching

```Ocaml
match expr with pattern-matching
```

* The `pattern-matching` part of the match is a sequence of clauses.
* Each one of the form: `pattern -> expr` separated by vertical bars `(|)`. 
* The clauses are processed in order, and only the expr of first matching clause is evaluated. 
* The value of the entire match expression is the value of the expr of the matching clause
* Match can be non-exhaustive can be result in the eval time exception `Match_failure` or a compile time warning. 

In [89]:
let newerf x =
    match x with
      "foo" -> "it is foo"
    | "bar" -> "bar is the one"
    | "zap" -> "totally different"

File "[89]", lines 2-5, characters 4-34:
2 | ....match x with
3 |       "foo" -> "it is foo"
4 |     | "bar" -> "bar is the one"
5 |     | "zap" -> "totally different"
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
""


val newerf : string -> string = <fun>


## Printing in Ocaml


* Printing "Hello World"
* Several of the built-in primitive types: `print_char`, `print_int`, `print_string`, and `print_float`
```Ocaml
print_string "Hello World from 3100"
```
* printing is a side effect
* Hence not functional programming style.

In [ ]:
print_string "hello"

## Type inference in mutually recursive functions.

We saw mutually recursive functions are a set of functions that can call each other.

```
let rec f1 v1 v2 ... vn =
  ...
and f2 v1 v2 ... vn =
   ...
...
and fn v1 v2 ... vn =
   ...
```

where the functions `f1` to `fn` can call each other. 

There was a question about how is type checking done.

## How is type inference done?

* Similar to solving equations in mathematics. 
* Initially, imagine the types of the individual components of the expression as unknowns. 
* The operators, argument type annotations etc, gives rise to constraints that these unknowns should satisfy. 
* For example, if the function contains `x+y`, then it immediately adds a constraint that the unknown type-of-x must be `int` and that type-of-y must be `int`.
* Another example : if there is a variable `z`, which the programmer annotated to be `z:int`. This adds the constraint that the initial unknown type-of-z must by `int`. 

* The type checker will solve the question : **is there a type assignment to the types which is consistent with the constraints?**. 
* If there is no consistent assigment of types to the unknown, then type error will be given.
* Type checker is simply checking if a system of constraints is satisfiable by some type assignments to the unknowns involved !!.

```OCaml
let rec even n =
  if n = 0 then true
  else odd (n-1)
  
and odd n = 
  if n = 0 then false 
  else even (n-1)
```
  
### Curiosity : How is type inference done?

* Mutually recursive functions - additional constraints for the type unknowns.
* The two functions give rise to constraints on what the type of the other should be. 
* This just adds to the above constraint satisfaction system. 